In [10]:
# File given on 28-11-2022 Backtest Results

In [11]:
!pip install quantstats as qs
!pip install numpy as np
!pip install pandas as pd
!pip install os
!pip install copy

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement copy (from versions: none)
ERROR: No matching distribution found for copy


In [12]:
import pandas as pd
import numpy as np
import os
import copy
import warnings
import quantstats as qs

qs.extend_pandas()

warnings.filterwarnings('ignore')

# function to run backtest using the input data and specified signal column
def backtest(df_data_spy, signal_column, cash_balance=10000, fee=0.01, max_allowed=1, save_logs=True):
    # lists to store the open and closed orders
    open_orders = []
    closed_orders = []

    # initialize columns for the logs to
    df_data_spy['signal'] = 0
    df_data_spy['portfolio_value'] = 0
    df_data_spy['cash_balance'] = cash_balance
    df_data_spy['position'] = 0

    # for each row in the data run the backtest considering the signal and the closing price
    for i in range(0, df_data_spy.shape[0]-1):
        close = df_data_spy['Close'].iloc[i]
        date = df_data_spy['Date/Time'].iloc[i]

        # ensure that the starting cash balance and the position are same as the previous timestamp
        if i > 0:
            df_data_spy['position'].iloc[i] = df_data_spy['position'].iloc[i-1]
            df_data_spy['cash_balance'].iloc[i] = df_data_spy['cash_balance'].iloc[i - 1]

        # signal from the specified signal column
        signal = df_data_spy[signal_column].iloc[i]

        # position
        position = df_data_spy['position'].iloc[i]

        # calculate the quantity (1+fee) * qty * price = available cash balance
        qty = (df_data_spy['cash_balance'].iloc[i]/(close)) / (1 + fee)

        # if signal is 1 and we have cash we buy
        if signal == 1 and (max_allowed == None or position <= max_allowed) and \
                df_data_spy['cash_balance'].iloc[i] >= qty * close * (1 + fee):
            # buy
            order = {"date": date, "buy_price": df_data_spy['Close'].iloc[i+1], "sell_price": None, "pnl": None,
                     "selling_date": None, "fee": None, "quantity": qty, 'net_pnl': None}
            order['fee'] = order['quantity'] * fee
            df_data_spy['position'].iloc[i] += qty
            df_data_spy['cash_balance'].iloc[i] -= qty * close - order['fee']
            open_orders.append(order)
        # if signal is either 0 or -1 we sell
        elif signal in [-1, 0] and position > 0:
            for order in open_orders:
                order['sell_price'] = close
                order['selling_date'] = date
                order['pnl'] = (order['sell_price'] - order['buy_price']) * order['quantity']
                order['fee'] += order['quantity'] * fee
                order['net_pnl'] = order['pnl'] - order['fee']
                df_data_spy['cash_balance'].iloc[i] += order['sell_price'] * order['quantity']  - order['fee']
                closed_orders.append(order)
            df_data_spy['position'].iloc[i] = 0
            open_orders = []
        # if signal is either 8,9 we sell 50% and close the remaining quantity
        elif signal in [8, 9] and position > 0:
            new_open_orders = []
            # sell 50%
            for order in open_orders:
                quantity = order['quantity']
                order['quantity'] = int(order['quantity']/2)
                order2 = dict(order)
                order['quantity'] = quantity - order['quantity']
                order['sell_price'] = close
                order['selling_date'] = date
                order['pnl'] = (order['sell_price'] - order['buy_price']) * order['quantity']
                order['fee'] += order['quantity'] * fee
                order['net_pnl'] = order['pnl'] - order['fee']
                df_data_spy['cash_balance'].iloc[i] += order['sell_price'] * order['quantity'] - order['fee']
                df_data_spy['position'].iloc[i] -= order['quantity']
                closed_orders.append(order)
                new_open_orders.append(order2)

            open_orders = copy.deepcopy(new_open_orders)



        df_data_spy['portfolio_value'].iloc[i] = close * df_data_spy['position'].iloc[i]

    # for the last day we close all the open positions
    df_data_spy['position'].iloc[-1] = df_data_spy['position'].iloc[-2]
    df_data_spy['cash_balance'].iloc[-1] = df_data_spy['cash_balance'].iloc[-2]
    close = df_data_spy['Close'].iloc[-1]
    for order in open_orders:
        order['sell_price'] = close
        order['pnl'] = (order['sell_price'] - order['buy_price']) * order['quantity']
        order['fee'] += order['quantity'] * fee
        df_data_spy['cash_balance'].iloc[-1] += order['sell_price'] * order['quantity'] * (1 - fee)
        closed_orders.append(order)

    df_data_spy['position'].iloc[-1] = 0
    df_data_spy['portfolio_value'].iloc[-1] = 0
    df_data_spy['total_equity'] = df_data_spy['cash_balance'] + df_data_spy['portfolio_value']

    trade_log = pd.DataFrame(closed_orders)
    if save_logs:
        trade_log.to_csv("trade_log.csv", index=False)
        df_data_spy.to_csv("backtest_log.csv", index=False)
        df_data_spy['total_equity'] = df_data_spy['cash_balance'] + df_data_spy['portfolio_value']
        returns = df_data_spy[['Date/Time', 'total_equity']]
        returns.rename(columns={'Date/Time':'Date'}, inplace=True)
        returns.set_index('Date', inplace=True)
        returns = returns['total_equity']
        returns = returns.pct_change()
        qs.reports.html(returns, "SPY", output='output.html')

    return

if __name__ == '__main__':
    path_to_data_folder = "data"
    signal_file_path = os.path.join(path_to_data_folder, "D:\\A Scott Internship\\Input Data for Lightgmb Classifier\\Signals for SPY 2000 to 102822.xlsx")
    data_file_path = os.path.join(path_to_data_folder, "D:\\A Scott Internship\\Input Data for Lightgmb Classifier\\ETF Ticker Data thru 102822.xlsx")

    df_signal = pd.read_excel(signal_file_path, sheet_name="Signals")
    df_signal['Date/Time'] = pd.to_datetime(df_signal['Date/Time'])
    print(df_signal.columns)

    df_data = pd.read_excel(data_file_path)
    df_data_spy = df_data.loc[df_data['Ticker'] == "SPY"].reset_index(drop=True)
    df_data_spy['Date/Time'] = pd.to_datetime(df_data_spy['Date/Time'])
    df_data_spy = pd.merge(df_data_spy, df_signal, on='Date/Time', how='left')

    backtest(df_data_spy, 'signal EMA 7 50')

Index(['Date/Time', 'signal RSIX.2575 NT ', 'signal 17 Day Top Channel Bucket',
       'signal 300 day Low ChannelBucket', 'signal 50 day Low ChannelPos',
       'signal 70 Day Buy Bot dips', 'signal buy BBbot',
       'signal Buy Dips> ATR X, HT 8day', 'signal BUY HOLD',
       'Signal C/DEMA150', 'Signal C/DEMA150.1', 'signal DV2E 595 ex50 S',
       'signal DVDS cust ', 'signal EMA 10 50', 'Signal EMA 10 70 ',
       'signal EMA 25x125', 'signal EMA 3 30', 'signal EMA 3/30 2Buff',
       'signal EMA 3/30 Sell Buff', 'signal EMA 40x200', 'Signal EMA 5 50 ',
       'Signal EMA 5X85', 'signal EMA 7 50', 'signal EMA110-60lag',
       'signal Fractal RSI B55 S35', 'signal FRAMA4/FRAMA5'],
      dtype='object')


In [13]:
df_data_spy

,Ticker,Date/Time,Open,High,Low,Close,Volume,signal RSIX.2575 NT,signal 17 Day Top Channel Bucket,signal 300 day Low ChannelBucket,...,Signal EMA 5X85,signal EMA 7 50,signal EMA110-60lag,signal Fractal RSI B55 S35,signal FRAMA4/FRAMA5,signal,portfolio_value,cash_balance,position,total_equity
0,SPY,2000-01-03,97.29011,97.29011,94.41898,95.44438,12440705,0,1,1,...,1,1,1,1,1,0,9900.990099,100.047258,103.735706,10001.037357
1,SPY,2000-01-04,94.17288,94.54203,91.62989,91.71192,12327182,1,0,1,...,1,1,1,1,-1,0,9513.800728,100.047258,103.735706,9613.847986
2,SPY,2000-01-05,91.83497,92.86037,90.07128,91.87598,18556602,0,0,1,...,1,1,1,1,-1,0,9530.819607,100.047258,103.735706,9630.866865
3,SPY,2000-01-06,91.62989,92.86037,90.39941,90.39941,9488965,1,0,1,...,1,1,1,1,-1,0,9377.646577,100.047258,103.735706,9477.693835
4,SPY,2000-01-07,92.08106,95.64946,91.91700,95.64946,12291830,0,1,1,...,1,1,1,1,-1,0,9922.264218,100.047258,103.735706,10022.311476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5739,SPY,2022-10-24,375.89001,380.06000,373.10999,378.87000,85436904,0,0,1,...,-1,-1,-1,-1,-1,0,0.000000,26382.639803,0.000000,26382.639803
5740,SPY,2022-10-25,378.79001,385.25000,378.67099,384.92001,78846352,0,0,1,...,-1,-1,-1,-1,-1,0,0.000000,26382.639803,0.000000,26382.639803
5741,SPY,2022-10-26,381.62000,387.57999,381.35001,382.01999,104087344,0,0,1,...,-1,-1,-1,9,-1,0,0.000000,26382.639803,0.000000,26382.639803
5742,SPY,2022-10-27,383.07001,385.00000,379.32999,379.98001,81971760,0,0,1,...,-1,-1,-1,1,1,0,0.000000,26382.639803,0.000000,26382.639803


In [20]:
df_data_spy["signal"]

0       0
1       0
2       0
3       0
4       0
       ..
5739    0
5740    0
5741    0
5742    0
5743    0
Name: signal, Length: 5744, dtype: int64

In [21]:
df_data_spy["cash_balance"]

0         100.047258
1         100.047258
2         100.047258
3         100.047258
4         100.047258
            ...     
5739    26382.639803
5740    26382.639803
5741    26382.639803
5742    26382.639803
5743    26382.639803
Name: cash_balance, Length: 5744, dtype: float64

In [18]:
open_orders

NameError: name 'open_orders' is not defined

In [17]:
closed_orders

NameError: name 'closed_orders' is not defined

In [22]:
df_data_spy['portfolio_value']

0       9900.990099
1       9513.800728
2       9530.819607
3       9377.646577
4       9922.264218
           ...     
5739       0.000000
5740       0.000000
5741       0.000000
5742       0.000000
5743       0.000000
Name: portfolio_value, Length: 5744, dtype: float64

In [23]:
df_data_spy['signal EMA 7 50"]

SyntaxError: EOL while scanning string literal (1835025164.py, line 1)

In [24]:
signal

NameError: name 'signal' is not defined

In [25]:
df_data_spy[signal_column]

NameError: name 'signal_column' is not defined